In [ ]:
# Project 1: Regression Analysis and Resampling Methods
# Part A: Ordinary Least Squares (OLS) for the Runge function

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
# Define the Runge function: f(x) = 1/(1+25x²)
def runge_function(x):
    """Runge function for polynomial fitting analysis."""
    return 1 / (1 + 25 * x**2)

# Generate dataset
def generate_dataset(n_points=100, noise_level=0.1, x_range=(-1, 1)):
    """Generate dataset for the Runge function with optional noise."""
    x = np.linspace(x_range[0], x_range[1], n_points)
    y_true = runge_function(x)
    
    # Add Gaussian noise
    noise = np.random.normal(0, noise_level, n_points)
    y_noisy = y_true + noise
    
    return x, y_true, y_noisy

# Test the function
x_test, y_true_test, y_noisy_test = generate_dataset(n_points=100, noise_level=0.1)

# Plot the original function
plt.figure(figsize=(10, 6))
plt.plot(x_test, y_true_test, 'b-', label='True Runge function', linewidth=2)
plt.plot(x_test, y_noisy_test, 'ro', label='Noisy data', markersize=4, alpha=0.7)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Runge Function: f(x) = 1/(1+25x²)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Polynomial features function (adapted from week 35)
def polynomial_features(x, degree, intercept=True):
    """Create polynomial features matrix."""
    n = len(x)
    if intercept:
        X = np.ones((n, degree + 1))
        for j in range(1, degree + 1):
            X[:, j] = x**j
    else:
        X = np.zeros((n, degree))
        for j in range(degree):
            X[:, j] = x**(j + 1)
    return X

# OLS parameters function using pseudoinverse
def ols_parameters(X, y):
    """Calculate OLS parameters using pseudoinverse."""
    return np.linalg.pinv(X) @ y

# MSE and R² calculation functions
def calculate_mse(y_true, y_pred):
    """Calculate Mean Squared Error."""
    return np.mean((y_true - y_pred)**2)

def calculate_r2(y_true, y_pred):
    """Calculate R² score."""
    y_mean = np.mean(y_true)
    ss_tot = np.sum((y_true - y_mean)**2)
    ss_res = np.sum((y_true - y_pred)**2)
    return 1 - (ss_res / ss_tot)

# Test the polynomial features function
x_small = np.array([-1, 0, 1])
X_test = polynomial_features(x_small, degree=3)
print("Polynomial features test:")
print("x =", x_small)
print("X (degree=3):")
print(X_test)


In [ ]:
# Part A: OLS Analysis with different polynomial degrees
# Generate larger dataset for analysis
n_points = 200
x, y_true, y_noisy = generate_dataset(n_points=n_points, noise_level=0.1)

# Split data into training and test sets (80/20 split)
x_train, x_test, y_train, y_test = train_test_split(x, y_noisy, test_size=0.2, random_state=42)

# Analyze polynomial degrees from 1 to 15
degrees = range(1, 16)
train_mses = []
test_mses = []
train_r2s = []
test_r2s = []
parameters = []

print("Polynomial Degree Analysis:")
print("Degree | Train MSE | Test MSE  | Train R²  | Test R²")
print("-" * 55)

for degree in degrees:
    # Create polynomial features
    X_train = polynomial_features(x_train, degree)
    X_test_poly = polynomial_features(x_test, degree)
    
    # Scale the features (important for higher degrees)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test_poly)
    
    # Calculate OLS parameters
    beta = ols_parameters(X_train_scaled, y_train)
    parameters.append(beta.copy())
    
    # Make predictions
    y_train_pred = X_train_scaled @ beta
    y_test_pred = X_test_scaled @ beta
    
    # Calculate metrics
    train_mse = calculate_mse(y_train, y_train_pred)
    test_mse = calculate_mse(y_test, y_test_pred)
    train_r2 = calculate_r2(y_train, y_train_pred)
    test_r2 = calculate_r2(y_test, y_test_pred)
    
    train_mses.append(train_mse)
    test_mses.append(test_mse)
    train_r2s.append(train_r2)
    test_r2s.append(test_r2)
    
    print(f"{degree:6d} | {train_mse:8.4f} | {test_mse:8.4f} | {train_r2:8.4f} | {test_r2:8.4f}")
